In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [ ]:
X = []
y = []

In [ ]:
background_folder = r"D:\git\PRACTICE\DATA\level_1_train\level_1\background"
for file in os.listdir(background_folder):
    img_path = os.path.join(background_folder, file)
    img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
    arr = img_to_array(img)/255.0
    X.append(arr)
    y.append(0)

In [ ]:
text_folders = [
    r"D:\git\PRACTICE\DATA\level_1_train\level_1\en",
    r"D:\git\PRACTICE\DATA\level_1_train\level_1\hi",
    r"D:\git\PRACTICE\DATA\level_1_train\level_1\ta"
]

In [ ]:
for folder in text_folders:
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
        arr = img_to_array(img)/255.0
        X.append(arr)
        y.append(1)

In [ ]:
X = np.array(X)
y = np.array(y)

X = 1 - X

In [ ]:
X = X.reshape(len(X), -1)
print("Training data shape:", X.shape, "Labels shape:", y.shape)

In [ ]:
X_test = []
test_ids = []

In [ ]:
test_folder = r"D:\git\PRACTICE\DATA\level_1_test\kaggle_level_1"
for file in os.listdir(test_folder):
    img_path = os.path.join(test_folder, file)
    img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
    arr = img_to_array(img)/255.0
    arr = 1 - arr  # invert
    X_test.append(arr)
    test_ids.append(int(file.split('.')[0]))

X_test = np.array(X_test).reshape(len(X_test), -1)
print("Test data shape:", X_test.shape)

In [ ]:
model = Sequential([
    Dense(128, input_shape=(256,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # output probability
])

model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.1, verbose=2)

In [ ]:
y_pred_prob = model.predict(X_test).flatten()
y_pred_class = (y_pred_prob >= 0.5).astype(int)

In [ ]:
df = pd.DataFrame({
    'ImageId': test_ids,
    'Probability': y_pred_prob,
    'Class': y_pred_class
})
df.to_csv(r"D:\git\PRACTICE\DATA\OUTPUT\ffnn_predictions.csv", index=False)
print("Predictions saved to ffnn_predictions.csv")

# Multi Class

In [ ]:
X = []
y = []

background_folder = r"D:\git\PRACTICE\DATA\level_1_train\level_1\background"
for file in os.listdir(background_folder):
    img_path = os.path.join(background_folder, file)
    img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
    arr = img_to_array(img)/255.0
    X.append(arr)
    y.append(0)

In [ ]:
text_classes = {
    1: r"D:\git\PRACTICE\DATA\level_1_train\level_1\en",
    2: r"D:\git\PRACTICE\DATA\level_1_train\level_1\hi",
    3: r"D:\git\PRACTICE\DATA\level_1_train\level_1\ta"
}

In [ ]:
for label, folder in text_classes.items():
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
        arr = img_to_array(img)/255.0
        X.append(arr)
        y.append(label)

In [ ]:
X = np.array(X)
y = np.array(y)

X = 1 - X

In [ ]:

X = X.reshape(len(X), -1)

y_cat = to_categorical(y, num_classes=4)

print("Training shape:", X.shape, "Labels shape:", y_cat.shape)

In [ ]:
X_test = []
test_ids = []

In [ ]:
test_folder = r"D:\git\PRACTICE\DATA\level_1_test\kaggle_level_1"
for file in os.listdir(test_folder):
    img_path = os.path.join(test_folder, file)
    img = load_img(img_path, target_size=(16,16), color_mode='grayscale')
    arr = img_to_array(img)/255.0
    arr = 1 - arr
    X_test.append(arr)
    test_ids.append(int(file.split('.')[0]))

X_test = np.array(X_test).reshape(len(X_test), -1)
print("Test shape:", X_test.shape)

In [ ]:
model = Sequential([
    Dense(128, input_shape=(256,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # multi-class output
])

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X, y_cat, epochs=100, batch_size=32, validation_split=0.1, verbose=2)

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred_class = np.argmax(y_pred_prob, axis=1)

In [ ]:
df = pd.DataFrame({
    'ImageId': test_ids,
    'Class': y_pred_class
})
df.to_csv(r"D:\git\PRACTICE\DATA\OUTPUT\MULTI_ffnn_predictions.csv", index=False)
print("Predictions saved to ffnn_multiclass_predictions.csv")